<a href="https://colab.research.google.com/github/andrebelem/Curso_GEE/blob/main/solu%C3%A7%C3%B5es_exerc%C3%ADcios_propostos/Solu%C3%A7%C3%A3o_Exerc%C3%ADcio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Possível solução do Exercício 1

In [1]:
# Bloco de inicialização do geemap e do ee
import ee
import geemap
ee.Authenticate()

In [2]:
# Aqui vocẽ deve trocar o projeto pelo "SEU-PROJECT-ID"
ee.Initialize(project='ee-andrebelem')

Agora sim a solução proposta...

In [3]:
# Solução
# Primeiro, precisamos saber exatamente como o FAO/GAUL está organizado. Procure o `table schema` deste dataset

# Carregando a FeatureCollection
dataset = ee.FeatureCollection("FAO/GAUL/2015/level1")
# uma maneira de listar o que precisamos é usando a função `ee.Reducer.toList()`
import pandas as pd

adm1 = dataset.reduceColumns(ee.Reducer.toList(), ['ADM1_NAME']).get('list').getInfo()
country = dataset.reduceColumns(ee.Reducer.toList(), ['ADM0_NAME']).get('list').getInfo()

list = pd.DataFrame({'state': adm1, 'country': country})
list[list['country'] == 'Brazil']

,state,country
847,Alagoas,Brazil
848,Bahia,Brazil
849,Ceara,Brazil
850,Espirito Santo,Brazil
851,Maranhao,Brazil
852,Minas Gerais,Brazil
853,Paraiba,Brazil
854,Pernambuco,Brazil
855,Piaui,Brazil
856,Rio De Janeiro,Brazil


In [4]:
# agora eu seleciono o estado que eu quero (São Paulo por exemplo) e crio o mapa

# Criando um mapa com geemap
map = geemap.Map()

# Carregando a FeatureCollection
dataset = ee.FeatureCollection("FAO/GAUL/2015/level1")
# agora vou filtra um nome:
estado = dataset.filter(ee.Filter.eq("ADM1_NAME", "Sao Paulo"))

# escolhendo as imagens e tirando a mediana
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterDate("2023-01-01", "2024-01-01")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5)) #<--- note que só selecionamos imagens com <5% de nuvens
)

# Calculando a mediana e recortando pela área do estado de São Paulo
image = collection.median().clip(estado) #<--- veja que estou dando um clip no objeto

vis = {
    "min": 0.0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"]
}
map.add_layer(image, vis, "Sentinel-2")
map.add_layer(estado, {}, "Estado de São Paulo")
map.center_object(estado, 7) #<--- note que vocẽ terá que ajustar o zoom de acordo com sua área
map

Map(center=[-22.26179743862149, -48.74090536767993], controls=(WidgetControl(options=['position', 'transparent…

Fim do Exercício 01.